In [31]:
import sys
sys.path.insert(0, 'C:\\Users\\Paarth Tandon\\Desktop\\repos\\personal-music-gen')

In [32]:
from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from personal_musicgen.model_utils import generate

import torch
import torchaudio

In [34]:
model = MusicGen.get_pretrained('small')
model.lm = model.lm.to(torch.float32)
model.lm.load_state_dict(torch.load('..\experimental_results\\eyedazzler\\eyedazzler_normal\\524_epochs\\checkpoint_524.pth')['model_state_dict'])

c:\users\paarth tandon\desktop\repos\personal-music-gen\libs\audiocraft\audiocraft\models\musicgen.py:116: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-small
  warnings.warn(
c:\Users\Paarth Tandon\Desktop\repos\personal-music-gen\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


<All keys matched successfully>

In [35]:
import torch
from audiocraft.utils.autocast import TorchAutocast
from audiocraft.modules.conditioners import ClassifierFreeGuidanceDropout
import numpy as np
import IPython.display as ipd
from scipy.io.wavfile import write


def generate_long_seq(
    model, description, num_samples, temp=1.0, top_k=250, top_p=0.0
):
    """Instead of using a text prompt, half the sample length of the previous generation is used as prompt input 

    Args:
        model (MusicGen): The pretrained MusicGen Model
        num_samples (integer): How many samples should be created, one sample is half the totgal_gen_len
        total_gen_len (integer): Maximum generation length.
        use_sampling (bool): Whether to use a sampling strategy or not.
        temp (float): Softmax temperature parameter. Defaults to 1.0.
        top_k (integer): top_k used for sampling. Defaults to 250.
        top_p (float): top_p used for sampling, when set to 0 top_k is used. Defaults to 0.0.
        cfg_coef (float): Coefficient used for classifier free guidance. Defaults to 3.0.

    Returns:
        List[torch.Tensor]: Output is a list of numpy vectors
    """
    samples = []
    prev_generated = None
    descriptions = [description]

    attributes, prompt_tokens = model._prepare_tokens_and_attributes(descriptions, None)
    model.generation_params = {
        'max_gen_len': int(30 * model.frame_rate),
        'use_sampling': True,
        'temp': temp,
        'top_k': top_k,
        'top_p': top_p,
        'cfg_coef': 3,
        'two_step_cfg': 0,
    }

    with model.autocast:
        total = []
        for _ in range(num_samples):
            with model.autocast:
                gen_tokens = model.lm.generate(prompt_tokens, attributes, callback=None, **model.generation_params)
                total.append(gen_tokens[..., prompt_tokens.shape[-1] if prompt_tokens is not None else 0:])
                prompt_tokens = gen_tokens[..., -gen_tokens.shape[-1] // 2:]
        gen_tokens = torch.cat(total, -1)

    with torch.no_grad():
        gen_audio = model.compression_model.decode(gen_tokens, None)
    gen_audio = gen_audio.cpu()
    return gen_audio

In [36]:
gen = generate(model, 'personal melodic guitar drums')
display_audio(gen[0], 32_000)

In [24]:
for i in range(20):
    gen = generate(model, 'personal')
    torchaudio.save(f'{i}.wav', gen[0], 32000)

KeyboardInterrupt: 

In [7]:
# prompt = 'personal'
# long_gen_audio = generate_long_seq(model, prompt, 4)
# torchaudio.save(f'long_{1}.wav', long_gen_audio[0], 32000)